# Examining Racial Discrimination in the US Job Market

### Background
Racial discrimination continues to be pervasive in cultures throughout the world. Researchers examined the level of racial discrimination in the United States labor market by randomly assigning identical résumés to black-sounding or white-sounding names and observing the impact on requests for interviews from employers.

### Data
In the dataset provided, each row represents a resume. The 'race' column has two values, 'b' and 'w', indicating black-sounding and white-sounding. The column 'call' has two values, 1 and 0, indicating whether the resume received a call from employers or not.

Note that the 'b' and 'w' values in race are assigned randomly to the resumes when presented to the employer.

### Exercises
You will perform a statistical analysis to establish whether race has a significant impact on the rate of callbacks for resumes.

Answer the following questions **in this notebook below and submit to your Github account**. 

   1. What test is appropriate for this problem? Does CLT apply?
   2. What are the null and alternate hypotheses?
   3. Compute margin of error, confidence interval, and p-value.
   4. Write a story describing the statistical significance in the context or the original problem.
   5. Does your analysis mean that race/name is the most important factor in callback success? Why or why not? If not, how would you amend your analysis?

You can include written notes in notebook cells using Markdown: 
   - In the control panel at the top, choose Cell > Cell Type > Markdown
   - Markdown syntax: http://nestacms.com/docs/creating-content/markdown-cheat-sheet


#### Resources
+ Experiment information and data source: http://www.povertyactionlab.org/evaluation/discrimination-job-market-united-states
+ Scipy statistical methods: http://docs.scipy.org/doc/scipy/reference/stats.html 
+ Markdown syntax: http://nestacms.com/docs/creating-content/markdown-cheat-sheet
****

In [93]:
import pandas as pd
import numpy as np
from scipy import stats

In [94]:
data = pd.io.stata.read_stata('data/us_job_market_discrimination.dta')

In [95]:
# number of callbacks for black-sounding names
sum(data[data.race=='b'].call)

157.0

In [96]:
data.head(10)

,id,ad,education,ofjobs,yearsexp,honors,volunteer,military,empholes,occupspecific,...,compreq,orgreq,manuf,transcom,bankreal,trade,busservice,othservice,missind,ownership
0,b,1,4,2,6,0,0,0,1,17,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,
1,b,1,3,3,6,0,1,1,0,316,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,
2,b,1,4,1,6,0,0,0,0,19,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,
3,b,1,3,4,6,0,1,0,1,313,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,
4,b,1,3,3,22,0,0,0,0,313,...,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,Nonprofit
5,b,1,4,2,6,1,0,0,0,266,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,Private
6,b,1,4,2,5,0,1,0,0,13,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,Private
7,b,1,3,4,21,0,1,0,1,313,...,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,Nonprofit
8,b,1,4,3,3,0,0,0,0,316,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,Private
9,b,1,4,2,6,0,1,0,0,263,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,Private


## 1. What test is appropriate for this problem? Does CLT apply?

   - Proportion test is the appropriate test for this problem 
   - CLT has 2 requirements. Namely: 
           Independence : sample observations are independent (for this problem, they are independent) 
           Sample Size  : There should be at least 10 successes (np>10) and 10 failures (n(1-p)>10) 

Because each row represents different resume, we can say that the sample observations are independent. 
Now we will check the secoond condition of sample size as follows:

In [97]:
black = data[data.race=='b']

white = data[data.race=='w']

len_black = len(black)

len_white = len(white)


calls_black = sum(black.call)

calls_white = sum(white.call)

#pooled Probability
Probability_called = sum(data.call)/len(data)
Probability_called

0.080492813141683772

In [98]:
# CHECK np and n(1-p) for White 
print('np (white):',len_white * Probability_called , '>10')

print('n(1-p) (white):',len_white * (1-Probability_called), '>10')

np (white): 196.0 >10
n(1-p) (white): 2239.0 >10


In [99]:
# CHECK  np and n(1-p) for Black t
print('np (black):',len_black * Probability_called, '>10')

print('n(1-p) (black):',len_black * (1-Probability_called), '>10')

np (black): 196.0 >10
n(1-p) (black): 2239.0 >10


### -> Since for both black and white, the n x p and n x (1-p) resulted in # greater than 10, the second requirement for CLT is satisfied

## 2. What are the null and alternate hypotheses?

   - H0: proportion of call for white = proportion of call for black         
   - H1: proportion of call for white != proportion of call for black
   
 OR we can rewrite this as:
 
 
   - H0: proportion of call for white - proportion of call for black = 0         
   - H1: proportion of call for white - proportion of call for black != 0

## 3. Compute margin of error, confidence interval, and p-value.


   - Margin of Error     :  0.0152819123109
   - Confidence Interval : (0.016750941898551489, 0.047314766520339682)
   - p-value             : 3.98388683759e-05

In [92]:
Standard_Error = (((Probability_called*(1-Probability_called))/len_black)+
                  ((Probability_called*(1-Probability_called))/len_white))**0.5

prob_white = calls_w/len_white

prob_black = calls_b/len_black

Z= (prob_white - prob_black)/Standard_Error

Margin_Error = 1.96*Standard_Error

CI = (prob_white-prob_black-Margin_Error,prob_white-prob_black+Margin_Error)

#pvalue compute
from scipy import stats
pvalue = stats.norm.sf(abs(Zstat))*2


print('Margin of Error:',Margin_Error)
print('Confidence Interval:',CI)
print('p-value:',pvalue)

Margin of Error: 0.0152819123109
Confidence Interval: (0.016750941898551489, 0.047314766520339682)
p-value: 3.98388683759e-05


## 4. Write a story describing the statistical significance in the context or the original problem.

Since the P-value (3.9*10^-5) is less than 0.05, we reject the Null hypothesis in favor of the alternative. Furthermore, this is and indication that race actually does have and have an impact on the rate of call backs for applicants. 

## 5. Does your analysis mean that race/name is the most important factor in callback success? Why or why not? If not, how would you amend your analysis?

No, we cannot make the conclusion that either race or name is the most important factor in the callback success. This is becauce we are only comparing the proportion of race in this analysis. In order to determine other factors that might have impact on the callback success, we would need to do a more complex analysis such as comparing the correlations of the other factors and the succress rate, performing logistic regression + lasso regression to see what factors are significant.